<a href="https://colab.research.google.com/github/yandexdataschool/MLatImperial2019/blob/master/07_lab/styletransfer_seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is basically slightly changed PyTorch official styletransfer
https://pytorch.org/tutorials/advanced/neural_style_tutorial.html

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy
import numpy as np

from IPython.display import clear_output

%matplotlib inline

In [0]:
!wget https://github.com/yandexdataschool/MLatImperial2019/raw/master/07_lab/images/chicago.jpg
!wget https://github.com/yandexdataschool/MLatImperial2019/raw/master/07_lab/images/picasso.jpg
!wget https://github.com/yandexdataschool/MLatImperial2019/raw/master/07_lab/images/rain_princess.jpg
!wget https://github.com/yandexdataschool/MLatImperial2019/raw/master/07_lab/images/my_img.jpg
!wget https://github.com/yandexdataschool/MLatImperial2019/raw/master/07_lab/images/shanghai.jpg

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [0]:
# desired size of the output image
imsize = 512 if torch.cuda.is_available() else 128  # use small size if no gpu

loader = transforms.Compose([
    transforms.Resize((imsize, imsize)),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = Image.open(image_name)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

In [0]:
# Set our content and style images

style_img = image_loader("./rain_princess.jpg")
content_img = image_loader("./shanghai.jpg")

assert style_img.size() == content_img.size(), \
    "we need to import style and content images of the same size"

In [0]:
unloader = transforms.ToPILImage()  # reconvert into PIL image

def imshow(tensor, title=None):
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated


plt.figure()
plt.axis("off")
imshow(style_img, title='Style Image')

plt.figure()
plt.axis("off")
imshow(content_img, title='Content Image')

In [0]:
class ContentLoss(nn.Module):

    def __init__(self, target, weight=1):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()
        self.weight = weight

    def forward(self, input):
        self.loss = <YOUR CODE>
        return input
      
    def backward(self, retain_graph=True):
      self.loss.backward(retain_graph=retain_graph)
      return self.loss

In [0]:
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise image to to matrix

    G = <YOUR CODE>  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

class StyleLoss(nn.Module):

    def __init__(self, target_feature, weight=1):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()
        self.weight = weight

    def forward(self, input):
        <YOUR CODE>
        self.loss = <YOUR CODE>
        return input
      
    def backward(self, retain_graph=True):
      self.loss.backward(retain_graph=retain_graph)
      return self.loss      

In [0]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

In [0]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

Define the layers to use, and add them from initial net to our new model

In [0]:
# desired depth layers to compute style/content losses :
content_layers_default = ['conv_4']
style_layers_default = ['conv_3', 'conv_4', 'conv_5']
style_weight = 1000000
content_weight = 1


def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)

    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target, content_weight)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature, style_weight)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

In [0]:
model, style_losses, content_losses = get_style_model_and_losses(cnn,
    cnn_normalization_mean, cnn_normalization_std, style_img, content_img)

In [0]:
# Note we **only** optimise picture gradients, not network weights
input_img = content_img.clone()
optimizer = optim.LBFGS([input_img.requires_grad_()])

In [0]:
num_steps=800

run = [0]
while run[0] <= num_steps:
    def closure():
        # correct the values of updated input image
        input_img.data.clamp_(0, 1)

        optimizer.zero_grad()
        model(input_img)
        style_score = 0
        content_score = 0

        for sl in style_losses:
            style_score += sl.backward()
        for cl in content_losses:
            content_score += cl.backward()
        
        run[0] += 1
        
        # Just fancy plotting and losses
        if run[0] % 30 == 0:
            clear_output()
            print("run {}:".format(run))
            print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                style_score.item(), content_score.item()))
            print()
            plt.figure(figsize=[10,10])
            plt.axis("off")
            imshow(input_img.data)
            plt.show()                

        return style_score + content_score
    
    # Note here, that since L-BFGS require iterations a trick
    # is used here. You can read about python closures by yourself.
    optimizer.step(closure)

# a last correction...
input_img.data.clamp_(0, 1);

In [0]:
plt.figure(figsize=(10, 10))
plt.axis("off")
imshow(input_img, title='Output Image')

sphinx_gallery_thumbnail_number = 4
plt.show()